<a href="https://colab.research.google.com/github/TrabalhosPUCPR/Python-TicTacToe/blob/master/TicTacToe_Jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jogo da Velha em Python

Implementação em python de um jogo da velha para qualquer tamanho de tabuleiro e inteligência artificial

Cada parte do codigo foi separado em diferentes blocos aqui no colab para facilitar a leitura de cada parte individual da implementação...

A implementação inteira é formada por:

- TicTacToe: Essa classe é responsável por armazenar os quadrados do campo em uma lista, setar um novo quadrado em um dos index desta lista e verificar se alguem ganhou e empatou, além de ter outras funções para informar sobre uma posição específica do tabuleiro

- TicTacToeGame: Essa classe é responsável por controlar o tabuleiro (classe TicTacToe), ela gerencia o que cada jogador é, recebe qual será a jogada dos jogadores e chama as funções do tabuleiro. De forma geral, ela controla o fluxo do jogo

- Player: Essa classe é a classe usada pra um jogador, ela possui as informações que um jogador teria, como o nome, e o seu símbolo, e possui a função act() que serve para pegar a coordenada escolhida pelo jogador para colocar no campo

- Ai: Essa classe possui as mesmas informações herdadas da classe Player (simbolo, nome, e a função act), então ela é do tipo player já que ela também retorna coordenadas para fazer uma jogada. Nessa classe possui toda a lógica para ela decidir qual a melhor jogada fazer, usando algoritmo minmax com poda alpha-beta, usando também uma heurística criada por uma função interna


Essas são as classes principais mais importantes para toda a implementação, para uma explicação com mais detalhes sobre cada um, você pode acessar a área de cada parte abaixo.

Uma breve explicação foi feita dentro do bloco de cada parte.

Pra jogar o jogo, você pode executar todas as células juntas, ou rodar apenas [o último bloco de código](#scrollTo=MZM4aMDWsM-_&line=1&uniqifier=1).


Clique no link para pular para partes do codigo:

- [Logger dos turnos](#scrollTo=gyzw3WI0sM-0)

- [Classe que possui a logica do campo](#scrollTo=_p80YRw3sM-8)

- [Classe do jogador humano](#scrollTo=EJN62o0YsM-3)

- [Classe da Inteligencia artificial](#scrollTo=qise7rUssM-5)

- [Classe controladora do campo do jogo](#scrollTo=pTZYwm3FsM--)

- [Execução do jogo](#scrollTo=MZM4aMDWsM-_&line=1&uniqifier=1)


## Classe para armazenar e imprimir informações sobre o turno

Classe para guardar os logs dos turnos. 

Ele guarda: 
- O tempo de inicio e fim de uma jogada em segundos
- Qual o numero do jogador que fez a jogada
- A coordenada que foi feita a jogada
- O estado do jogo após a jogada
- O número total de jogadas

E na hora que ele é convertido para string, ele passa todas essas informações

In [2]:
from datetime import datetime, timedelta

class TurnLogger:
    def __init__(self):
        self.logs = []
        self.start_time = datetime.now()
        self.end_time = datetime.now()
        self.player_n_turn = 0
        self.latest_placed_coord = (0, 0)
        self.game_state = TurnState.Continue
        self.total_turns = 0

    def restart_timer(self):
        self.start_time = datetime.now()

    def end_timer(self):
        self.end_time = datetime.now()

    def elapsed_time(self) -> timedelta:
        return self.end_time - self.start_time

    def __str__(self):
        string = f"Turn: {self.total_turns}\nPlayer: {self.player_n_turn}\nPlaced coordinates: x={self.latest_placed_coord[0]}; y={self.latest_placed_coord[1]}\nGame State: {TurnState.to_string(self.game_state)}\nElapsed Time: {self.elapsed_time().total_seconds()}"
        self.logs.append(string)
        return string


## Classe que controla o logica backend do jogo

### Square State

Essa classe realmente é meio desnecessária, mas como no rust eu usei um enum que guardava um char qnd era to tipo filled, eu trouxe algo parecido pra essa versaoem python, mas nessa versao e quase igual a so usa 0,1,2 pra representa cada coisa


De qlqr maneira, essa classe e instancializada toda vez e a gente usa ela observandoa variavel symbol, que vai ser None caso estiver vazio, ou guardar o simbolo que foi posicionado nela como um character

In [3]:
class SquareState:
    def __init__(self, symbol=None):
        self.symbol = symbol

    def is_none(self):
        return self.symbol is None

    def __eq__(self, other):
        return self.symbol == other.symbol

### Turn State

Turn state e so um enumerador pra representa um número que é o estado de jogo de forma que possa ser lido claramente, to to_string apenas pega em string cada um destes estados

In [4]:
class TurnState:
    Continue = 0
    Draw = 1
    Error = 2
    Victory = 3

    @staticmethod
    def to_string(turn_state):
        if turn_state == TurnState.Continue:
            return "Continue"
        elif turn_state == TurnState.Victory:
            return "Victory"
        elif turn_state == TurnState.Draw:
            return "Draw"
        else:
            return "Error"

### TicTacToe

Aqui que está a lógica real do campo do jogo da velha, ela guarda:
- O tamanho do campo 

- Quantos quadrados seguidos será nescessário para alguém ganhar o jogo

- A lista com seus quadrados

- os caracteres a serem usados para desenhar na tela

- Quantos quadrados foram preenchidos

- O desenho do campo em formato de string para qualquer tamanho e qualquer preenchimento deste

E tem as funções:
- get_square / get_square_by_index: esta funcao recebe tanto um index ou uma coordenada de um quadrado do campo e retorna o SquareState dela, vazio ou nao

- get_coord_index / get_index_coord: funcao helper apenas para converter um index para coordenada e vice-versa

- set_square / set_square_from_index: funcao que serve para setar o SquareState passado como parametro na coordenada/index dado, ela retorna qual o estado do jogo após fazer isso, esse estado pode ser qualquer um, incluindo o estado de erro

- check_game_over: responsável por ler os quadrados em volta do index dado como parâmetro e verificar se a partir do index houve alguma vitória, ou se teve empate

- check_draw: verifica se houve empate, para fazer isso ele apenas verifica se a quantidade de quadrados preenchidos é igual ou não o tamanho total do campo

- check_x / y / left_diag / right_diag: verifica quantos quadrados iguais ao do SquareState passado como parametro existe na sua linha/coluna/diagonais, estas funções são apenas contadores.  
Ele basicamente verifica casa por casa que está ao lado do index dado, e verifica se é igual ou não, caso for, ele continua, se não, ele faz a mesma coisa, só que para o outro lado, então para o check_x(), ele verifica o lado direito 1 por 1, quando a distancia desta checagem igualar a distancia da sequencia minima para ganhar o jogo, ou ele encontrar um SquareState que não for igual, ele quebra este loop, e recomeça do lado esquerdo.   
Estas funções também possuem diferentes modos:
    - stop_counting: quando for True: para de contar caso encontrar um SquareState vazio ou com um simbolo diferente, quando for False: para de contar apenas se encontrar um simbolo do oponente, NÃO quando for vazio
    - return_available_spaces: conta a quantidade de espaçoes disponíveis o SquareState tem e retorna

O campo também possui algumas funções utilitárias que é especialmente útil pra criar a heurística do IA com uma maior quantidade de informações, estas são:

- sum_squares_in_winnable_distance: conta quantos quadrados iguais ao do passado como parâmetro estão em uma distancia mínima para vitória a partir do index dado, esta função tem 2 modos dependendo se o return_highest é verdadeiro ou não:
    - Verdadeiro: retorna apenas a soma de uma linha que possui a maior quantidade de quadrados iguais em uma distancia de vitória
    - Falso: retorna a soma de quadrados iguais em todas as direções

- check_n_of_available_axis: retorna a quantidade de direções que a partir do index dado como parâmetro é possível obter vitória

- spaces_of_around: verifica quantos quadrados em volta do index é igual ao SquareState passado como parâmetro, a maneira que ele faz isso é pegando primeiro os quadrados da direita e esquerda, depois verifica o de cima e de baixo da esquerda para a direita


In [5]:
class TicTacToe:

    def __init__(self, x_size=3, y_size=3, seq_to_win=3, empty_symbol=' ', horizontal_spacer='─', vertical_spacer='│',
                 cross_spacer="┼"):
        self.x_size = x_size
        self.y_size = y_size
        self.squares = [SquareState(None) for _ in range(x_size * y_size)]
        self.seq_to_win = seq_to_win
        self.filled = 0
        self.empty_symbol = empty_symbol
        self.horizontal_spacer = horizontal_spacer
        self.cross_spacer = cross_spacer
        self.vertical_spacer = vertical_spacer

    def __str__(self):
        def add_col_margin(s, col_margin):
            s += " " * col_margin
            return s

        def add_line_margin(s, line_margin):
            s += " " * (line_margin - 1)
            return s

        s = "\n"
        lane = 1
        col_margin = len(str(self.y_size)) + 4
        if col_margin % 2 == 0:
            col_margin += 1
        line_margin = len(str(self.x_size))
        col = 1
        s = add_col_margin(s, col_margin // 2)
        for i in range(self.x_size * 3):
            if i % 3 == 0:
                s += f" {col}"
                col += 1
            else:
                s += " "
        s += f"\n{lane}-"
        s = add_line_margin(s, line_margin)
        for i in range(len(self.squares)):
            if self.squares[i].is_none():
                s += f" {self.empty_symbol} "
            else:
                s += f" {self.squares[i].symbol} "
            if (i > 0) and ((i + 1) % self.x_size == 0) and not (lane == self.y_size):
                s += "\n"
                s = add_col_margin(s, col_margin // 2)
                for i in range(self.x_size * 3 + self.x_size - 1):
                    if (i + 1) % 4 == 0:
                        s += self.cross_spacer
                    else:
                        s += self.horizontal_spacer
                lane += 1
                s += f"\n{lane}-"
                s = add_line_margin(s, line_margin)
                if len(str(lane + 1)) > len(str(lane)):
                    line_margin -= 1
            elif i < len(self.squares) - 1:
                s += self.vertical_spacer
        return s

    def get_square(self, x: int, y: int) -> SquareState:
        return self.squares[self.get_coord_index(x, y)]

    def get_coord_index(self, x, y) -> int:
        return int(x + self.x_size * y)

    def get_index_coord(self, index):
        return index % self.x_size, int(index / self.x_size)

    def set_square(self, x: int, y: int, state: SquareState):
        i = self.get_coord_index(x, y)
        return self.set_square_from_index(i, state)

    def set_square_from_index(self, index: int, square_state: SquareState):
        if index >= len(self.squares) or not self.squares[index].is_none():
            return TurnState.Error
        self.squares[index] = square_state
        self.filled += 1
        coord = self.get_index_coord(index)
        return self.check_game_over(coord[0], coord[1], square_state)

    def check_game_over(self, x: int, y: int, state: SquareState):
        if self.filled >= self.seq_to_win + 2 and \
                ((self.check_x(x, y, state, True, False) >= self.seq_to_win) or
                 (self.check_y(x, y, state, True, False) == self.seq_to_win) or
                 (self.check_left_diag(x, y, state, True, False) == self.seq_to_win) or
                 (self.check_right_diag(x, y, state, True, False) == self.seq_to_win)):
            return TurnState.Victory
        if self.check_draw():
            return TurnState.Draw
        return TurnState.Continue

    def check_x(self, x: int, y: int, state: SquareState, stop_counting: bool, return_available_spaces: bool) -> int:
        available_spaces_count = 1
        seq_count = 1
        for i in [1, -1]:
            dist = i
            while 0 <= x + dist < self.x_size and dist * i  < self.seq_to_win:
                square_state = self.get_square(x + dist, y)
                if not square_state.is_none():
                    if square_state == state:
                        seq_count += 1
                    else:
                        break
                elif stop_counting:
                    break
                dist += i
                available_spaces_count += 1
        if return_available_spaces:
            return available_spaces_count
        elif not stop_counting or seq_count >= self.seq_to_win:
            return seq_count
        else:
            return 1

    def check_y(self, x: int, y: int, state: SquareState, stop_counting: bool, return_available_spaces: bool) -> int:
        available_spaces_count = 1
        seq_count = 1
        for i in [1, -1]:
            dist = i
            while 0 <= y + dist < self.y_size and dist * i  < self.seq_to_win:
                square_state = self.get_square(x, y + dist)
                if not square_state.is_none():
                    if square_state == state:
                        seq_count += 1
                    else:
                        break
                elif stop_counting:
                    break
                dist += i
                available_spaces_count += 1
        if return_available_spaces:
            return available_spaces_count
        elif not stop_counting or seq_count >= self.seq_to_win:
            return seq_count
        else:
            return 1

    def check_left_diag(self, x: int, y: int, state: SquareState, stop_counting: bool, return_available_spaces: bool) -> int:
        available_spaces_count = 1
        seq_count = 1
        for i in [1, -1]:
            dist = i
            while 0 <= y + dist < self.y_size and 0 <= x + dist < self.x_size and dist * i < self.seq_to_win:
                square_state = self.get_square(x + dist, y + dist)
                if not square_state.is_none():
                    if square_state == state:
                        seq_count += 1
                    else:
                        break
                elif stop_counting:
                    break
                dist += i
                available_spaces_count += 1
        if return_available_spaces:
            return available_spaces_count
        elif not stop_counting or seq_count >= self.seq_to_win:
            return seq_count
        else:
            return 1

    def check_right_diag(self, x: int, y: int, state: SquareState, stop_counting: bool, return_available_spaces: bool) -> int:
        available_spaces_count = 1
        seq_count = 1
        for i in [1, -1]:
            dist = i
            while 0 <= y + (dist*-1) < self.y_size and 0 <= x + dist < self.x_size and dist * i < self.seq_to_win:
                square_state = self.get_square(x + dist, y + dist*-1)
                if not square_state.is_none():
                    if square_state == state:
                        seq_count += 1
                    else:
                        break
                elif stop_counting:
                    break
                dist += i
                available_spaces_count += 1
        if return_available_spaces:
            return available_spaces_count
        elif not stop_counting or seq_count >= self.seq_to_win:
            return seq_count
        else:
            return 1

    def check_draw(self):
        return self.filled == len(self.squares)

    def size(self):
        return self.x_size * self.y_size

    def get_square_by_index(self, index):
        return self.squares[index]

    def clear(self):
        self.squares = [SquareState() for _ in range(self.x_size * self.y_size)]
        self.filled = 0

    def sum_squares_in_winnable_distance(self, index: int, square_state: SquareState, return_highest=False) -> int:
        x, y = self.get_index_coord(index)
        if return_highest:
            count = [0, 0, 0, 0]
            count[0] = self.check_x(x, y, square_state, False, False) - 1
            count[1] = self.check_y(x, y, square_state, False, False) - 1
            count[2] = self.check_left_diag(x, y, square_state, False, False) - 1
            count[3] = self.check_right_diag(x, y, square_state, False, False) - 1
            highest = count[0]
            for i in range(1, len(count)):
                if count[i] > highest:
                    highest = count[i]
            return int(highest)
        else:
            return int((self.check_x(x, y, square_state, False, False) +
                        self.check_y(x, y, square_state, False, False) +
                        self.check_left_diag(x, y, square_state, False, False) +
                        self.check_right_diag(x, y, square_state, False, False)) - 4)

    def check_n_of_available_axis(self, index: int, square_state: SquareState) -> int:
        coord = self.get_index_coord(index)
        axis = 0
        if self.check_x(coord[0], coord[1], square_state, False, True) >= self.seq_to_win:
            axis += 1
        if self.check_y(coord[0], coord[1], square_state, False, True) >= self.seq_to_win:
            axis += 1
        if self.check_left_diag(coord[0], coord[1], square_state, False, True) >= self.seq_to_win:
            axis += 1
        if self.check_right_diag(coord[0], coord[1], square_state, False, True) >= self.seq_to_win:
            axis += 1
        return axis

    def spaces_of_around(self, index: int, square_state: SquareState) -> int:
        count = 0
        x, y = self.get_index_coord(index)
        if x - 1 >= 0:
            if square_state == self.get_square(x-1, x):
                count += 1
        if x + 1 < self.x_size:
            if square_state == self.get_square(x+1, y):
                count += 1
        if x != 0:
            check_x = x - 1
            spaces_to_check = 3
        else:
            check_x = 0
            spaces_to_check = 2
        for i in range(spaces_to_check):
            if check_x + i < self.x_size:
                if y - 1 >= 0:
                    if square_state == self.get_square(check_x+i, y - 1):
                        count += 1
                if y + 1 < self.y_size:
                    if square_state == self.get_square(check_x+i, y + 1):
                        count += 1
        return count


## Classe dos jogadores

### Jogador humano

Classe que pega como input a coluna e a linha que o jogador quer jogar, guarda:
- O nome do jogador
- O simbolo do jogador

Sua única função é o act() que apenas devolve a coluna e a linha para jogar

In [6]:
import abc


class Player:
    def __init__(self, name, square_symbol):
        self.name = name
        self.square_symbol = square_symbol

    @abc.abstractmethod
    def act(self, current_board: TicTacToe):
        while True:
            try:
                col = int(input(f"{self.name}'s turn, type the column of your next move\ncolumn: "))-1
                line = int(input(f"line: "))-1
            except ValueError:
                print("Type a valid number")
                continue
            if 0 < col < current_board.x_size and 0 < line < current_board.y_size and \
                    not current_board.get_square(col, line).is_none():
                print("Invalid column or line number")
                continue
            break
        return col, line


### Inteligencia Artificial



#### Node

Essa classe não é tão nescessária assim também, é apenas uma forma mais fácil de representar um valor, que possui uma heurística ou um score

In [7]:
class Node:
    def __init__(self, data, data_score=0.0):
        self.data = data
        self.data_score = data_score
        self.children = []

    def __lt__(self, other):
        return self.data_score < other.data_score

    def __le__(self, other):
        return self.data_score <= other.data_score

    def __gt__(self, other):
        return self.data_score > other.data_score

    def __ge__(self, other):
        return self.data_score >= other.data_score

    def is_terminal(self):
        return len(self.children) == 0


#### Logica do IA

Aqui está a racíocinio que o IA usa para decidir a jogada, ele herda da classe [Player](#scrollTo=EJN62o0YsM-3&line=1&uniqifier=1), então possui um nome, um simbolo, e a função act()

Geral de como funciona: recebe o campo atual, e faz uma jogada para toda posição possivel e guarda em uma lista, para cada uma destas jogadas, ele da uma pontuação (score) baseado no quão boa é esta jogada, com tudo isso feito, ele corda esta lista caso o tamanho da lista exceda o tamanho máximo de filhos do IA e retorna esta lista esta lista entao é iterada, fazendo de maneira recursiva, a mesma coisa para todas as outras jogadas não ultrapassando o tamanho máximo de camadas do IA, quando isto termina, o IA vai começar a analizar qual a melhor jogada que cada jogador fará considerando que os dois estão jogando de forma ótima usando minmax com poda alpha-beta. Basicamente escolhendo qual jogada que levará para o melhor final.

Algoritmo minmax não precisa ser explicado aqui, MAS a heurística precisa:

O QUE FORMA A HEURISTÍCA (score):
- attack_score: quantos quadrados da pessoa jogando estão em uma distância de vitória a partir da posição dada: Ex.![image_2023-05-31_003758351.png](https://media.discordapp.net/attachments/1113311058803830794/1113311076440866916/image.png?width=299&height=237)

  Vamos dizer que o X em vermelho é a posição que está sendo analisada, conseguimos ver que tanto verticalmente, e horizontalmente, possui um X E estes dois estão com o potencial para ganhar o jogo, ja que possui espaço suficiente a partir deles para pegar 4 X seguidos, então o score de ataque final desta posição, é 2. Mas agora vamos analisar outro exemplo:![image_2023-05-31_003758351.png](https://media.discordapp.net/attachments/1113311058803830794/1113311298512506970/image.png?width=316&height=271)

  Aqui, podemos observar que a partir do X vermelho, possuem outros 2 X na mesma diagonal, MAS, estes não estão em uma distância de vitória! Então não são contados, a mesma coisa aconteceria por exemplo, caso os X estivessem em uma distância MAIOR que a sequência mínima para ganhar. Então neste exemplo, o score de ataque, seria 0.

- available_axis: quantidade de direções que a posição pode usar para ganhar o jogo, usando como exemplo a primeira imagem deste bloco:  
A partir do X vermelho ele possui 2 eixos que podem ser usados para ganhar o jogo, verticalmente, e horizontalmente.  
Mas agora olhando o segundo exemplo:
Apenas 1 eixo está disponível, horizontalmente, pois verticalmente e diagonal está sendo bloqueado pelo oponente

- total_defense_score: funciona da mesma maneira que o attack_score, a diferença é: ele usa o símbolo do oponente, desta forma ele consegue contar quantos símbolos que poderiam estar ligados ao oponente, se ele tivesse colocado neste index

- empty_space_around_score: é apenas quantidade de espaços vazios que está em volta, CASO, o index estiver nas posições centrais das colunas e linhas nas pontas do campo, este valor é zerado

Estas heurísticas também possuem prioridades, sendo que se o attack score for maior que outra jogada por exemplo, esta jogada terá prioridade

atack_score possui prioridade normal

total_defense_score e available_axis possui prioridade menor, então os dois são divididos por 10

empty_spaces_around possui a menor prioridade, então é dividida por 100

A heuristica final, é a soma de tudo isto divididos por 100, para ter menor prioridade de uma jogada que resulta em uma vitória

Caso a jogada sendo analisada for uma jogada feito pelo IA, a heurística é positiva, caso for uma jogada do oponente, a heurística é negativa

In [8]:
import copy


class Ai(Player):

    def __init__(self, player: Player, max_node_childs, max_layers, op_symbol):
        super().__init__(player.name, player.square_symbol)
        self.max_node_childs = max_node_childs
        self.max_layers = max_layers
        self.op_symbol = op_symbol

    def act(self, current_board):
        root = Node((current_board, 0, TurnState.Continue))
        _, index = self.compute_moves(root, float('-inf'), float('inf'), self.max_layers, True)
        return current_board.get_index_coord(index)

    def compute_moves(self, current_node, alpha, beta, layer, maximizing):
        if layer == 0 or current_node.data[2] != TurnState.Continue:
            return current_node.data_score, current_node.data[1]
        if maximizing:
            best_move = (float('-inf'), 0)
            for move in self.get_possible_moves(current_node.data[0], maximizing):
                childs_best, _ = self.compute_moves(move, alpha, beta, layer-1, False)
                if best_move[0] < childs_best:
                    best_move = (childs_best, move.data[1])
                alpha = max(alpha, childs_best)
                if beta <= alpha:
                    break
            return best_move
        else:
            best_move = (float('inf'), 0)
            for move in self.get_possible_moves(current_node.data[0], maximizing):
                childs_best, _ = self.compute_moves(move, alpha, beta, layer-1, True)
                if best_move[0] > childs_best:
                    best_move = (childs_best, move.data[1])
                beta = min(beta, childs_best)
                if beta <= alpha:
                    break
            return best_move

    def get_possible_moves(self, current_board: TicTacToe, maximizing):
        moves = []
        for index, square in enumerate(current_board.squares):
            if square.is_none():
                if maximizing:
                    square_state = SquareState(self.square_symbol)
                else:
                    square_state = SquareState(self.op_symbol)
                board_copy = copy.deepcopy(current_board)
                possible_move_node = Node((board_copy, index, board_copy.set_square_from_index(index, square_state)))
                if possible_move_node.data[2] == TurnState.Victory:
                    if maximizing:
                        possible_move_node.data_score = 1.0
                    else:
                        possible_move_node.data_score = -1.0
                elif possible_move_node.data[2] == TurnState.Continue:
                    possible_move_node.data_score = self.get_move_heuristic(possible_move_node.data[0], square_state,
                                                                            index, maximizing)
                moves.append(possible_move_node)
        moves.sort(key=lambda x: x.data_score, reverse=maximizing)
        if len(moves) > 0 and len(moves) > self.max_node_childs:
            moves = moves[0:self.max_node_childs]
        return moves

    def get_move_heuristic(self, board: TicTacToe, square_state, index, maximizing) -> float:
        attack_score = board.sum_squares_in_winnable_distance(index, square_state)
        available_axis = board.check_n_of_available_axis(index, square_state)
        if maximizing:
            op_state = SquareState(self.op_symbol)
        else:
            op_state = SquareState(self.square_symbol)
        total_defense_score = board.sum_squares_in_winnable_distance(index, op_state)
        empty_space_around_score = board.spaces_of_around(index, SquareState())
        if board.seq_to_win**2 < board.size():
            highest_defense_score = board.sum_squares_in_winnable_distance(index, op_state, return_highest=True)
            if (board.seq_to_win % 2 != 0 and highest_defense_score >= (board.seq_to_win / 2.0).__ceil__()) or \
                    (board.seq_to_win % 2 == 0 and highest_defense_score >= (board.seq_to_win - 2)):
                total_defense_score *= 100
        coord = board.get_index_coord(index)
        if coord[1] == 0 or coord[1] == board.y_size-1:
            if 0 < coord[0] < board.x_size-1:
                empty_space_around_score = 0
        elif coord[0] == 0 or coord[0] == board.x_size-1:
            empty_space_around_score = 0
        total_defense_score /= 10.0
        heuristic = ((attack_score + total_defense_score + (available_axis/10.0)) +
                     (empty_space_around_score / 100.0)) / 100.0
        if maximizing:
            return heuristic
        else:
            return -heuristic


## Classe para o controle do jogo

### GameState

Mais uma classe que faz muito mais sentido ter no Rust do que no python, possui um enumerador representando o estado do jogo, e pode tambem ser instânciada, guardando este enumerador, e as informacoes do próximo jogador, junto com o seu número

In [9]:
class GameState:
    Begin = 0
    PlayerTurn = 1
    Finished = 2

    def __init__(self, state, next_player: Player = None, next_player_n=None):
        self.state = state
        self.next_player = (next_player, next_player_n)

### TicTacToeGame

Classe que controla o campo, ela guarda:
- Os dois jogadores, estes são da classe [Player](#scrollTo=EJN62o0YsM-3&line=1&uniqifier=1)
- O [campo](#scrollTo=BYtvFemqrZ9_&line=1&uniqifier=1)
- O estado do jogo

As funções são:
- setup_new_game_with_prompts: uma função apenas para configurar o jogo com prompts no terminal, mudando a dificuldade do IA, tamanho do campo, jogo com 1 player, 2, ou apenas com IA, etc.
- load_x_game: retorna um jogo com a configuracao baseado em qual destas funções foi chamado 
- start_game: comeca o loop do jogo, pegando as coordenadas de cada jogador no seu turno, mudando, fazendo um log de cada turno, e printando mensagens baseado no estado do jogo, além de imprimir o campo
- create_ai: apenas instância o [IA](#scrollTo=qise7rUssM-5&line=3&uniqifier=1) com as configurações baseado na dificuldade selecionada
- change_size: muda o tamanho do campo

In [19]:
from google.colab import output
from time import sleep

TITLE = """
  _______     _______      _______
 |__   __|   |__   __|    |__   __|
    | |   _  ___| | __ _  ___| | ___   ___
    | |  | |/ __| |/ _` |/ __| |/ _ \\ / _ \\
    | |  | | (__| | (_| | (__| | (_) |  __/
    |_|  |_|\\___|_|\\__,_|\\___|_|\\___/ \\___|"""
AUTHOR = "Gabrielle, Kovalski and Leonardo"
REPO_LINK = "https://github.com/TrabalhosPUCPR/PythonTicTacToe"


class AiDifficulties: # enumerador para representar o nivel de dificuldade do ai, apenas limitando ainda mais a quantidade de camadas e jogadas para analisar
    Easy = 0,
    Medium = 1,
    Hard = 2


class TicTacToeGame:
    def __init__(self, player1=Player("Player1", 'X'), player2=Player("Player2", 'O'), show_turn_info=True, overwrite_print=True):
        self.player1 = player1
        self.player2 = player2
        self.board = TicTacToe()
        self.show_turn_info = show_turn_info
        self.game_state = GameState(GameState.Begin, next_player=player1, next_player_n=1)
        self.overwrite_print = overwrite_print

    def __str__(self):
        if self.game_state.state == GameState.Begin or self.game_state.state == GameState.PlayerTurn:
            return f"\n{self.game_state.next_player[0].name}'s turn!\n\n{self.board.__str__()}\n"
        if self.game_state.state == GameState.Finished:
            return f"\nGame is finished!\n\n{self.board.__str__()}\n"

    @staticmethod
    def setup_new_game_with_prompts():
        while True:
            print(f"{TITLE}\n\nMade by {AUTHOR}\n\nRepo link: {REPO_LINK}\n\n")
            try:
                choice = int(input("\nPlease choose an option:\n1-Load 1 player game\n2-Load 2 player game\n3-Load Ai "
                                   "game\n4-Exit\n(default: 1): "))
            except ValueError:
                choice = 1

            if choice == 2:
                game = TicTacToeGame.load_default_2player_game()
            elif choice == 3:
                game = TicTacToeGame.load_default_ai_game(AiDifficulties.Hard, AiDifficulties.Hard)
            elif choice == 4:
                return
            else:
                try:
                    dif = int(input("\nChoose a difficulty for the AI\n1-Easy\n2-Medium\n3-Hard\n(default: 3): "))
                except ValueError:
                    dif = 3
                game = TicTacToeGame.load_default_1player_game(dif - 1)
            output.clear()
            print(f"\nBoard size: {game.board.x_size}x{game.board.y_size}\nSequence to win: {game.board.seq_to_win}")
            try:
                choice = int(input("\n1-Start Game\n2-Configure Game\n(default: 1): "))
            except ValueError:
                choice = 1
            if choice == 2:
                output.clear()
                while True:
                    sleep(0.1)
                    print(f"\nBoard size: {game.board.x_size}x{game.board.y_size}\nSequence to win: {game.board.seq_to_win}")
                    try:
                        choice = int(input("\n1-Change board size\n2-Debug Mode\n3-Start Game\n(default: 3): "))
                    except ValueError:
                        choice = 3
                    if choice == 1:
                        size = int(input("\nType the new board size: "))
                        seq = int(input("\nType sequence length to win: "))
                        if not game.change_size(size, seq):
                            print("\nInvalid size or sequence length!")
                            continue
                    elif choice == 2:
                        if game.show_turn_info:
                            print("\nDebug Mode Deactivated!")
                            game.show_turn_info = False
                            continue
                        else:
                            print("\nDebug Mode Activated!")
                            game.show_turn_info = True
                            continue
                    else:
                        output.clear()
                        return game
                    output.clear()
            else:
                break
        return game

    @staticmethod
    def load_default_1player_game(ai_difficulty):
        game = TicTacToeGame()
        game.player2 = TicTacToeGame.create_ai(game.player2, game.player1.square_symbol, ai_difficulty)
        return game

    @staticmethod
    def load_default_2player_game():
        return TicTacToeGame()

    @staticmethod
    def load_default_ai_game(ai1_difficulty, ai2_difficulty):
        game = TicTacToeGame()
        game.player1 = TicTacToeGame.create_ai(game.player1, game.player2.square_symbol, ai1_difficulty)
        game.player2 = TicTacToeGame.create_ai(game.player2, game.player1.square_symbol, ai2_difficulty)
        game.game_state = GameState(GameState.Begin, next_player=game.player1, next_player_n=1)
        return game

    def start_game(self):
        while True:
          if self.overwrite_print:
              output.clear()
          turn_logger = TurnLogger()
          print(self)
          while self.game_state.state == GameState.Begin or self.game_state.state == GameState.PlayerTurn:
              turn_logger.restart_timer()
              x, y = self.game_state.next_player[0].act(self.board)
              turn_logger.end_timer()
              board_state = self.board.set_square(x, y, SquareState(symbol=self.game_state.next_player[0].square_symbol))
              turn_logger.total_turns += 1
              turn_logger.latest_placed_coord = (x, y)
              turn_logger.player_n_turn = self.game_state.next_player[1]
              turn_logger.game_state = board_state
              if self.overwrite_print:
                  output.clear()
              if board_state == TurnState.Draw:
                  print(f"\nAll spaces hae been filled! It's a draw!")
                  self.game_state.state = GameState.Finished
              elif board_state == TurnState.Victory:
                  print(f"\n{self.board.seq_to_win} in a row! {self.game_state.next_player[0].name} wins!")
                  self.game_state.state = GameState.Finished
              elif board_state == TurnState.Error:
                  print(f"\nPlease type a valid postition!\n\n{self}")
                  continue
              else:
                  self.game_state.state = GameState.PlayerTurn
                  if self.game_state.next_player[1] == 1:
                      self.game_state.next_player = (self.player2, 2)
                  else:
                      self.game_state.next_player = (self.player1, 1)
              if self.show_turn_info:
                  print(f"\nPrevious turn info:\n{turn_logger}")
              print(self)
          input("Press enter to exit...")
          break

    @staticmethod
    def create_ai(player, op_symbol, difficulty):
        if difficulty == AiDifficulties.Easy:
            max_childs = 7
            max_layers = 1
        elif difficulty == AiDifficulties.Medium:
            max_childs = 6
            max_layers = 2
        else:
            max_childs = 10
            max_layers = 5
        return Ai(player, max_childs, max_layers, op_symbol)

    def change_size(self, size, seq_to_win):
        if size < seq_to_win:
            return False
        self.board.x_size = size
        self.board.y_size = size
        self.board.seq_to_win = seq_to_win
        self.board.squares = [SquareState() for _ in range(size**2)]
        return self


## Criando o jogo, configurando e rodando

- [Voltar para cima](#scrollTo=zV8AU5CLsM-x&line=17&uniqifier=1)

In [21]:
game = TicTacToeGame.setup_new_game_with_prompts()
if game is not None:
  game.start_game()


  _______     _______      _______
 |__   __|   |__   __|    |__   __|
    | |   _  ___| | __ _  ___| | ___   ___
    | |  | |/ __| |/ _` |/ __| |/ _ \ / _ \
    | |  | | (__| | (_| | (__| | (_) |  __/
    |_|  |_|\___|_|\__,_|\___|_|\___/ \___|

Made by Gabrielle, Kovalski and Leonardo

Repo link: https://github.com/TrabalhosPUCPR/PythonTicTacToe



Please choose an option:
1-Load 1 player game
2-Load 2 player game
3-Load Ai game
4-Exit
(default: 1): 4
